LOAD dataset and filter out  for wine



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder


data_wine = load_wine()
features, labels = data_wine.data, data_wine.target


encoder = OneHotEncoder(sparse_output=False) # 0 and 1
targets_encoded = encoder.fit_transform(labels.reshape(-1, 1))

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [ ]:



X_train, X_test, Y_train, Y_test = train_test_split(features_scaled, targets_encoded, test_size=0.2, random_state=42)


input_neurons = X_train.shape[1]
hidden_neurons = 10
output_neurons = Y_train.shape[1]


np.random.seed(42)
weights_1 = np.random.randn(input_neurons, hidden_neurons)
bias_1 = np.zeros((1, hidden_neurons))
weights_2 = np.random.randn(hidden_neurons, output_neurons)
bias_2 = np.zeros((1, output_neurons))

def activation_sigmoid(x):
    return 1 / (1 + np.exp(-x))

def activation_softmax(x):
    exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)

def loss_mse(actual, predicted):
    return np.mean((actual - predicted) ** 2)

def forward_pass(input_data, actual_output=None): # Add actual_output as an optional parameter
    z_hidden = np.dot(input_data, weights_1) + bias_1
    activation_hidden = activation_sigmoid(z_hidden)
    z_output = np.dot(activation_hidden, weights_2) + bias_2
    activation_output = activation_softmax(z_output)
    loss = loss_mse(actual_output, activation_output) if actual_output is not None else None
    return z_hidden, activation_hidden, z_output, activation_output, loss
def backward_pass(input_data, actual_output, z_hidden, activation_hidden, activation_output, learning_rate=0.1):
    global weights_1, bias_1, weights_2, bias_2
    error_output = activation_output - actual_output
    dW2 = np.dot(activation_hidden.T, error_output) / input_data.shape[0]
    dB2 = np.sum(error_output, axis=0, keepdims=True) / input_data.shape[0]

    error_hidden = np.dot(error_output, weights_2.T) * (activation_hidden * (1 - activation_hidden))
    dW1 = np.dot(input_data.T, error_hidden) / input_data.shape[0]
    dB1 = np.sum(error_hidden, axis=0, keepdims=True) / input_data.shape[0]

     # i have updated the wegiht based onthe graiedent desceit
    weights_1 -= learning_rate*dW1
    bias_1 -= learning_rate * dB1
    weights_2 -= learning_rate * dW2
    bias_2 -= learning_rate * dB2


epochs = 1000
loss_history = []
for epoch in range(epochs):
    z_hidden, activation_hidden, z_output, activation_output, loss = forward_pass(X_train, actual_output=Y_train) # Pass Y_train
    loss_history.append(loss)
    backward_pass(X_train, Y_train, z_hidden, activation_hidden, activation_output, learning_rate=0.1)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, MSE Loss: {loss:.4f}")


Plot training loss

In [ ]:

plt.figure(figsize=(8, 5))
plt.plot(loss_history, label='loss Curve', color='blue')
plt.xlabel("epochs")
plt.ylabel("mse Loss")
plt.title("training loss over epochs")
plt.legend()
plt.savefig("training_loss.png")
plt.show()


_, _, _, predicted_output, _ = forward_pass(X_test)
predicted_labels = np.argmax(predicted_output, axis=1)
true_labels = np.argmax(Y_test, axis=1)
accuracy = np.mean(predicted_labels == true_labels)
print(f"test Accuracy: {accuracy:.4f}")





visualize and compare


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(weights_1, aspect='auto', cmap='coolwarm')
axes[0].set_title("Forward Only - Initial Weights")
axes[1].imshow(weights_2, aspect='auto', cmap='coolwarm')
axes[1].set_title("Forward + Backward - Updated Weights")
plt.savefig("comparison.png")
plt.show()